<a href="https://colab.research.google.com/github/vin-thomas/BBC_news_classification/blob/main/Copy_of_Bert_bbc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import torch
import sklearn
from IPython.core.display import display, HTML
import glob
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 32.7 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 3.3 MB 46.4 MB/s 
     |████████████████████████████████| 596 kB 57.2 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import transformers
from transformers import BertTokenizer

In [4]:
URL = "http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip" 
!wget -P 'Data/' $URL

--2022-01-26 14:25:04--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874078 (2.7M) [application/zip]
Saving to: ‘Data/bbc-fulltext.zip’

bbc-fulltext.zip    100%[===================>]   2.74M  4.53MB/s    in 0.6s    

2022-01-26 14:25:05 (4.53 MB/s) - ‘Data/bbc-fulltext.zip’ saved [2874078/2874078]



In [ ]:
!unzip "/content/Data/bbc-fulltext.zip" -d 'Data/'

In [302]:
file_name_iter = glob.iglob('/content/Data/bbc/**/*.txt')

In [303]:
def get_topic(file):
  return file.split('/')[-2]

In [304]:
def get_content(file):
  with open(file, 'rb') as f:
    content = f.read()
    return content

In [305]:
labels = []
content = []

for file in file_name_iter:
  labels.append(get_topic(file))
  content.append (get_content(file))

In [308]:
mod_content =[]
for item in content:
  item= str(item)
  mod_content.append(item)


In [309]:
mod_content[0]

'b"Gritty return for Prince of Persia\\n\\nStill basking in the relatively recent glory of last year\'s Sands Of Time, the dashing Prince of Persia is back in Warrior Within, and in a more bellicose mood than last time.\\n\\nThis sequel gives the franchise a grim, gritty new look and ramps up the action and violence. As before, you control the super-athletic prince from a third-person perspective. The time-travelling plot hinges on the Dahaka, an all-consuming monster pursuing our hero through the ages. The only way to dispel it is to turn back the clock again and kill the sultry Empress Of Time before she ever creates the Sands of Time that caused the great beast\'s creation.\\n\\nStudiously structured though this back story is, everything boils down to old-fashioned fantasy gameplay which proves, on the whole, as dependable as it needs to be. Ever since the series\' then-groundbreaking beginnings on the Commodore Amiga, Prince of Persia has always been about meticulously-animated acr

In [310]:
type(mod_content), type(mod_content[0]), len(mod_content), type(labels), len(labels),type(labels[0])

(list, str, 2225, list, 2225, str)

In [311]:
labels, uniques = pd.factorize(labels)

In [312]:
type(labels)

numpy.ndarray

In [313]:
bbc_df = pd.DataFrame(list(zip(mod_content, labels)), columns =['Text', 'Labels'])

In [314]:
bbc_df.head(5)

,Text,Labels
0,"b""Gritty return for Prince of Persia\n\nStill ...",0
1,b'Broadband fuels online change\n\nFast web ac...,0
2,b'More women turn to net security\n\nOlder peo...,0
3,b'Hotspot users gain free net calls\n\nPeople ...,0
4,b'Mobile games come of age\n\nThe BBC News web...,0


In [315]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [316]:
class BBCDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer):
    self.df = df
    self.x = self.df['Text']
    self.y = self.df['Labels']
    self.n = df.shape[0]
    self.tokenizer = tokenizer

  def __len__(self):
    print (self.n)
    return self.n

  def __getitem__(self, index):
    
    text = str(self.x[index])
    text = " ".join(text.split())
    y = self.y[index]

    inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

    return {
        'input_ids': inputs['input_ids'].flatten(),
        'attention_mask': inputs['attention_mask'].flatten(),
        'token_type_ids': inputs["token_type_ids"].flatten(),
        'targets': torch.tensor(y)
        }


In [317]:
train_dataset = BBCDataset(bbc_df, tokenizer)

In [318]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=50,
    shuffle=True,
    num_workers=0
)

2225
2225


In [319]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [320]:
device

device(type='cpu')

In [321]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased', return_dict=True)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask, token_type_ids):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask, 
      token_type_ids=token_type_ids
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [322]:
model = BertClassifier(5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [323]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()

In [324]:
epochs=1

model.train()


for e in range(epochs):
  i=0
  for batch in (train_data_loader):
    i+=1
    input_ids = batch['input_ids'].to(device, dtype= torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype= torch.long)
    labels = batch['targets'].to(device, dtype= torch.float)

    print (input_ids.shape, attention_mask.shape, labels.shape)
             
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids= token_type_ids)
    loss = criterion (outputs, labels)
    
    if i % 100 == 0:
      print("loss - {0}, iteration - {1}/{2}".format(loss, e + 1, i))
    
    model.zero_grad()
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(),1)
    optimizer.step()

2225
torch.Size([50, 128]) torch.Size([50, 128]) torch.Size([50])


IndexError: ignored